In [72]:
import pandas as pd
import numpy as np
fe=pd.read_csv('50_Startups.csv',encoding= 'unicode_escape')

In [73]:
fe.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [74]:
fe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   R&D Spend        50 non-null     float64
 1   Administration   50 non-null     float64
 2   Marketing Spend  50 non-null     float64
 3   State            50 non-null     object 
 4   Profit           50 non-null     float64
dtypes: float64(4), object(1)
memory usage: 2.1+ KB


In [75]:
#note state has dtype object
#we can either convert it to float type but for now we will omit the state column
fe.drop("State", axis=1, inplace=True)

In [76]:
predict_variable='Profit'
fe2=fe.drop("Profit",axis=1,inplace=False)
other_variable=fe2.columns.tolist()


In [77]:
X=fe[predict_variable].values
Y=fe[other_variable].values.tolist()

In [78]:
#taking 80% data for creating the model and rest to cheak efiiciency
import math
Y_train=Y[0:math.floor(0.8*len(Y))]
Y_test=Y[math.floor(0.8*len(Y)):len(Y)]
X_train=X.tolist()[0:math.floor(0.8*len(X))]
X_test=X.tolist()[math.floor(0.8*len(Y)):len(X)]

In [79]:
from sklearn.preprocessing import MinMaxScaler
scale=MinMaxScaler()
Y_train=scale.fit_transform(Y_train).tolist()
Y_test=scale.transform(Y_test).tolist()

In [80]:
i=0
train=[]
while i<len(Y_train):
    train.append(Y_train[i]+[X_train[i]])
    i=i+1

In [81]:
#y=a0+a1x1+a2x2+....+anxn
#f=[a0,a1,a2,...,an]--line eqn
#p=[x1,x2,x3,..xn]--point
def evaluate(p,f):
    i=1
    S=f[0]
    while i<len(f):
        S=S+f[i]*p[i-1]
        i=i+1
    return(S)
#C is point C=[p1,p2,....,pn]
def deviation(p,f):
    return((p[-1]-evaluate(p,f))**2)
#A=[[G1,G2,G3,.....Gm]] Gi is point of the form C
#B is line eqn here B=[a0,a1,a2,......,an-1]
def totaldeviation(A,f):
    i=0
    S=0
    while i<len(A):
        S=S+deviation(A[i],f)
        i=i+1
    return(S/len(A))

#cost function.

#derivative cost
def der(f,A,i):
    if i==0:
        m=len(A)
        j=0
        S=0
        while j<m:
            S=S+1/m*(-A[j][len(A[j])-1]+evaluate(A[j],f))
            j=j+1
        return(S)
    else:
        m=len(A)
        j=0
        S=0
        while j<m:
            S=S+1/m*(-A[j][len(A[j])-1]+evaluate(A[j],f))*A[j][i-1]
            j=j+1
        return(S)

#der([0,2],[[1,3],[2,1],[6,7]],1)
#x2=0+2x1
#der([a,b],[[1,3],[2,1],[6,7]],1)
#x2=a+bx1
#(a+b-3)^2+(a+2b-1)^2+(a+6b-7)^2
    #(a+b-3)+2(a+2b-1)+6(a+6b-7)    


def gd(A,lr=0.1,ne=10000):
    theta= [0] * len(A[0])
    m=len(A)
    i=0
    D=[]
    while i<ne:
        k=0
        while k< len(theta):
            Q=der(theta,A,k)
            theta[k]=theta[k]-lr*Q
            k=k+1
        i=i+1
    return(theta)

In [82]:
f=gd(train)#these sre cofficients of linear function that best suits the given data

In [83]:
f

[70186.75083988544, 110974.65070315555, -6827.16123612345, 12991.190486829535]

In [84]:
X_pred=[]
i=0
while i<len(Y_test):
    X_pred=X_pred+[evaluate(Y_test[i],f)]
    i=i+1

In [85]:
len(X_pred)

10

In [86]:
meanX_test=sum(X_test)/len(X_test)
i=0
E1=0
while i<len(X_test):
    E1=E1+(X_test[i]-X_pred[i])**2
    i=i+1
j=0
E2=0
while j<len(X_test):
    E2=E2+(X_test[j]-meanX_test)**2
    j=j+1
r2=1-E1/E2

r2

In [87]:
r2

0.6006689723067702